In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import f1_score
import re
import warnings
import math

# Ignora avvisi non critici
warnings.filterwarnings('ignore')

# --- CONFIGURAZIONE HARDWARE ---
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device in uso: {device}")

# --- PARAMETRI GLOBALI ---
# 1. Definiamo le colonne
JOINT_COLS = [f'joint_{i:02d}' for i in range(30)]
SURVEY_COLS = ['pain_survey_1', 'pain_survey_2', 'pain_survey_3', 'pain_survey_4']
STATIC_COLS = []  # SVUOTATO (Data Analysis: erano cheat code inaffidabili)
TIME_COL = 'time'

# 2. Iperparametri Architettura & Training
WINDOW_SIZE = 40        # (Advice 11/11)
STRIDE = 10
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-4
EPOCHS = 50
GRADIENT_CLIP_VALUE = 1.0 # (Advice 10/11)
K_FOLDS = 5
LABEL_SMOOTHING = 0.1     # (Advice 09/11)

# --- 1. CARICAMENTO E PULIZIA ---
print("--- 1. Caricamento e Pulizia Iniziale ---")
try:
    df_features_raw = pd.read_csv('/kaggle/input/pirate/pirate_pain_train.csv')
    df_labels_raw = pd.read_csv('/kaggle/input/pirate/pirate_pain_train_labels.csv')
    df_test_raw = pd.read_csv('/kaggle/input/pirate/pirate_pain_test.csv')
except:
    # Fallback per path locale
    df_features_raw = pd.read_csv('pirate_pain_train.csv')
    df_labels_raw = pd.read_csv('pirate_pain_train_labels.csv')
    df_test_raw = pd.read_csv('pirate_pain_test.csv')

# Gestione Text Column (Team Name)
exclude_cols = ['label', 'sample_index']
string_cols = df_features_raw.select_dtypes(include=['object']).columns.tolist()
string_cols = [c for c in string_cols if c not in exclude_cols]

TEXT_COL = None
TEXT_VOCAB_SIZE = 0

if len(string_cols) > 0:
    TEXT_COL = string_cols[0] 
    print(f"Trovata colonna 'Team Name': {TEXT_COL}")
    
    def clean_team_name(text):
        if pd.isna(text): return "unknown"
        return re.sub(r'[^a-z0-9]', '', str(text).lower())

    df_features_raw[TEXT_COL] = df_features_raw[TEXT_COL].apply(clean_team_name)
    df_test_raw[TEXT_COL] = df_test_raw[TEXT_COL].apply(clean_team_name)
    
    le_text = LabelEncoder()
    all_text = pd.concat([df_features_raw[TEXT_COL], df_test_raw[TEXT_COL]], axis=0)
    le_text.fit(all_text)
    
    df_features_raw[TEXT_COL] = le_text.transform(df_features_raw[TEXT_COL])
    df_test_raw[TEXT_COL] = le_text.transform(df_test_raw[TEXT_COL])
    
    TEXT_VOCAB_SIZE = len(le_text.classes_)
else:
    print("Nessuna colonna 'Team Name' trovata.")

# --- 2. FEATURE ENGINEERING (Advice 12/11 + Delta) ---
def engineer_features(df):
    df_eng = df.copy()
    grouped = df_eng.groupby('sample_index')
    
    # A. Delta (Velocità)
    for col in JOINT_COLS:
        df_eng[f'd_{col}'] = grouped[col].diff().fillna(0)
    
    # B. Time Cyclicity (Advice 12/11)
    # Normalizziamo il tempo su un ciclo presunto (es. max del dataset o fisso 180)
    max_time_val = df_eng[TIME_COL].max() + 1 
    df_eng['sin_time'] = np.sin(2 * np.pi * df_eng[TIME_COL] / max_time_val)
    df_eng['cos_time'] = np.cos(2 * np.pi * df_eng[TIME_COL] / max_time_val)

    # C. Drop Costante
    if 'joint_30' in df_eng.columns:
        df_eng = df_eng.drop(columns=['joint_30'])
        
    return df_eng

print("Calcolo Feature Engineering...")
df_features_engineered = engineer_features(df_features_raw)
df_test_engineered = engineer_features(df_test_raw)

# Definizione Colonne Continue Finali
DELTA_JOINT_COLS = [f'd_{col}' for col in JOINT_COLS]
CONTINUOUS_COLS = JOINT_COLS + DELTA_JOINT_COLS + ['sin_time', 'cos_time']

# Vocabolari per Embedding (Advice 07/11)
survey_vocab_sizes = [int(df_features_engineered[c].max() + 1) for c in SURVEY_COLS]
time_vocab_size = int(df_features_engineered[TIME_COL].max() + 1)

# Mappatura Label
label_mapping = {'no_pain': 0, 'low_pain': 1, 'high_pain': 2}
df_labels_raw['label_encoded'] = df_labels_raw['label'].map(label_mapping)

# --- 3. DATASET & SAMPLER ---
class PiratePainDataset(Dataset):
    def __init__(self, features_df, labels_df, sample_indices, window_size, stride, text_col=None, augment=False):
        self.features_df = features_df
        self.labels_df = labels_df.set_index('sample_index') if labels_df is not None else None
        self.sample_indices = sample_indices
        self.window_size = window_size
        self.stride = stride
        self.text_col = text_col
        self.augment = augment 
        
        self.grouped_features = dict(tuple(features_df.groupby('sample_index')))
        self.indices = self._create_indices()

    def _create_indices(self):
        indices = []
        for sample_idx in self.sample_indices:
            if sample_idx not in self.grouped_features: continue
            data = self.grouped_features[sample_idx]
            n_timesteps = len(data)
            for start in range(0, n_timesteps - self.window_size + 1, self.stride):
                indices.append((sample_idx, start, start + self.window_size))
        return indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        sample_idx, start, end = self.indices[idx]
        window_data = self.grouped_features[sample_idx].iloc[start:end]

        # 1. Continui + Augmentation
        vals = window_data[CONTINUOUS_COLS].values
        if self.augment:
            noise = np.random.normal(0, 0.02, vals.shape) 
            vals = vals + noise
        x_cont = torch.tensor(vals, dtype=torch.float)
        
        # 2. Embeddings (Survey + Time)
        x_survey = torch.tensor((window_data[SURVEY_COLS].values + 1), dtype=torch.long)
        x_time = torch.tensor((window_data[TIME_COL].values + 1), dtype=torch.long)
        
        # 3. Text
        x_text = torch.tensor(0, dtype=torch.long)
        if self.text_col:
            val = window_data[self.text_col].iloc[0]
            x_text = torch.tensor(val, dtype=torch.long)

        # 4. Label
        label = torch.tensor(-1, dtype=torch.long)
        if self.labels_df is not None:
            label = torch.tensor(self.labels_df.loc[sample_idx, 'label_encoded'], dtype=torch.long)

        # Nota: Static rimosso
        return x_cont, x_survey, x_time, x_text, label

def get_weighted_sampler(dataset, labels_df):
    sample_to_label = labels_df.set_index('sample_index')['label_encoded'].to_dict()
    label_counts = labels_df['label_encoded'].value_counts().sort_index()
    class_weights = 1.0 / label_counts
    
    weights = []
    for idx_tuple in dataset.indices:
        s_idx = idx_tuple[0]
        if s_idx in sample_to_label:
            l = sample_to_label[s_idx]
            weights.append(class_weights[l])
        else:
            weights.append(0)
    return WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)

# --- 4. LOSS FUNCTION (Advice 09/11) ---
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean', label_smoothing=0.0):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.label_smoothing = label_smoothing

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(
            inputs, targets, reduction='none', weight=self.alpha, 
            label_smoothing=self.label_smoothing
        )
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        return focal_loss.mean() if self.reduction == 'mean' else focal_loss.sum()

# --- 5. MODELLO CNN-LSTM (Advice 13/11) ---
class PiratePainModel(nn.Module):
    def __init__(self, n_continuous, survey_vocab_sizes, time_vocab_size, text_vocab_size, lstm_hidden=128, n_classes=3):
        super().__init__()
        
        # Embeddings
        self.emb_surveys = nn.ModuleList([nn.Embedding(v+2, 4) for v in survey_vocab_sizes])
        self.emb_time = nn.Embedding(time_vocab_size+2, 8)
        
        self.use_text = (text_vocab_size > 0)
        text_dim = 8 if self.use_text else 0
        if self.use_text:
            self.emb_text = nn.Embedding(text_vocab_size+2, 8)
            
        # Dimensione Input Totale
        total_survey_dim = len(survey_vocab_sizes) * 4
        input_dim = n_continuous + total_survey_dim + 8 + text_dim
        
        # CNN Block (Feature Extraction)
        self.cnn = nn.Sequential(
            nn.Conv1d(in_channels=input_dim, out_channels=64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2)
        )
        
        # LSTM Block (Temporal Memory)
        self.lstm = nn.LSTM(128, lstm_hidden, num_layers=2, batch_first=True, dropout=0.3)
        
        # Classifier
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(lstm_hidden, n_classes)

    def forward(self, x_cont, x_survey, x_time, x_text):
        batch_size, seq_len, _ = x_cont.shape
        
        # Embeddings
        e_surv = [emb(x_survey[:,:,i]) for i, emb in enumerate(self.emb_surveys)]
        e_time = self.emb_time(x_time)
        
        # Concatenazione Base
        features = [x_cont] + e_surv + [e_time]
        if self.use_text:
            e_txt = self.emb_text(x_text).unsqueeze(1).repeat(1, seq_len, 1)
            features.append(e_txt)
            
        full_input = torch.cat(features, dim=2) # (B, Seq, Feat)
        
        # CNN Pass (Requires Permute: B, Feat, Seq)
        x = full_input.permute(0, 2, 1)
        x = self.cnn(x)
        
        # LSTM Pass (Requires Permute back: B, Seq, Feat)
        x = x.permute(0, 2, 1)
        out, _ = self.lstm(x)
        
        # Classify last step
        logits = self.classifier(self.dropout(out[:, -1, :]))
        return logits

# --- 6. TRAINING & VALIDATION UTILS ---
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for xc, xs, xt, xtxt, y in loader:
        xc, xs, xt, xtxt, y = xc.to(device), xs.to(device), xt.to(device), xtxt.to(device), y.to(device)
        optimizer.zero_grad()
        logits = model(xc, xs, xt, xtxt)
        loss = criterion(logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP_VALUE)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# --- 7. MAIN LOOP (K-FOLD + ENSEMBLE) ---
print("\n--- Avvio K-Fold con Ensemble Strategy ---")

all_sample_indices = df_labels_raw['sample_index'].unique()
all_labels_strat = df_labels_raw.set_index('sample_index').loc[all_sample_indices]['label_encoded'].values

# Setup per OOF
oof_probs = np.zeros((len(all_sample_indices), 3))
oof_targets = np.zeros(len(all_sample_indices))
sample_to_idx = {s: i for i, s in enumerate(all_sample_indices)}
models_list = [] 

skf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(all_sample_indices, all_labels_strat)):
    print(f"\n--- Fold {fold+1}/{K_FOLDS} ---")
    
    train_samples = all_sample_indices[train_idx]
    val_samples = all_sample_indices[val_idx]
    
    # Scaling
    scaler = StandardScaler()
    train_subset = df_features_engineered[df_features_engineered['sample_index'].isin(train_samples)]
    scaler.fit(train_subset[CONTINUOUS_COLS])
    
    df_fold = df_features_engineered.copy()
    df_fold[CONTINUOUS_COLS] = scaler.transform(df_fold[CONTINUOUS_COLS])
    
    # Dataset
    train_ds = PiratePainDataset(df_fold, df_labels_raw, train_samples, WINDOW_SIZE, STRIDE, TEXT_COL, augment=True)
    val_ds = PiratePainDataset(df_fold, df_labels_raw, val_samples, WINDOW_SIZE, STRIDE, TEXT_COL, augment=False)
    
    sampler = get_weighted_sampler(train_ds, df_labels_raw)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, shuffle=False, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
    
    # Init
    model = PiratePainModel(
        n_continuous=len(CONTINUOUS_COLS), 
        survey_vocab_sizes=survey_vocab_sizes, 
        time_vocab_size=time_vocab_size,
        text_vocab_size=TEXT_VOCAB_SIZE
    ).to(device)
    
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    criterion = FocalLoss(alpha=None, gamma=2.0, label_smoothing=LABEL_SMOOTHING)
    
    best_v_f1 = 0
    best_model_wts = None
    
    # Epochs
    for ep in range(EPOCHS):
        t_loss = train_epoch(model, train_loader, optimizer, criterion)
        
        # Validazione OOF
        model.eval()
        val_logits_list, val_sample_indices_list = [], []
        window_sample_map_val = [x[0] for x in val_ds.indices]
        
        with torch.no_grad():
            for xc, xs, xt, xtxt, y in val_loader:
                xc, xs, xt, xtxt = xc.to(device), xs.to(device), xt.to(device), xtxt.to(device)
                logits = model(xc, xs, xt, xtxt)
                val_logits_list.extend(logits.cpu().numpy())
        
        # Soft Voting
        df_val_logits = pd.DataFrame(val_logits_list, columns=[0, 1, 2])
        df_val_logits['sample_index'] = window_sample_map_val
        df_val_probs = df_val_logits.groupby('sample_index').mean()
        
        current_val_probs = torch.softmax(torch.tensor(df_val_probs.values), dim=1).numpy()
        current_val_preds = np.argmax(current_val_probs, axis=1)
        
        current_val_indices = df_val_probs.index
        current_val_labels = df_labels_raw.set_index('sample_index').loc[current_val_indices]['label_encoded'].values
        
        v_f1 = f1_score(current_val_labels, current_val_preds, average='weighted')
        
        if v_f1 > best_v_f1:
            best_v_f1 = v_f1
            best_model_wts = model.state_dict()
            # Save OOF
            for idx, s_idx in enumerate(current_val_indices):
                global_idx = sample_to_idx[s_idx]
                oof_probs[global_idx] = current_val_probs[idx]
                oof_targets[global_idx] = current_val_labels[idx]

    print(f"Fold {fold+1} Best Val F1: {best_v_f1:.4f}")
    model.load_state_dict(best_model_wts)
    models_list.append(model)

# --- 8. OTTIMIZZAZIONE SOGLIE ---
print("\n--- Ricerca Soglie Ottimali su OOF ---")
best_thresh = (0.0, 0.0)
best_score = 0.0

for t_high in np.arange(0.15, 0.50, 0.01):
    for t_low in np.arange(0.20, 0.55, 0.01):
        if t_low >= t_high: continue
        
        preds = []
        for p in oof_probs:
            if p[2] > t_high: preds.append(2)
            elif p[1] > t_low: preds.append(1)
            else: preds.append(0)
            
        s = f1_score(oof_targets, preds, average='weighted')
        if s > best_score:
            best_score = s
            best_thresh = (t_low, t_high)

print(f"Soglie Trovate: Low>{best_thresh[0]:.2f}, High>{best_thresh[1]:.2f} -> OOF F1: {best_score:.4f}")

# --- 9. INFERENZA FINALE ---
print("\n--- Generazione Submission (Ensemble) ---")
final_scaler = StandardScaler()
final_scaler.fit(df_features_engineered[CONTINUOUS_COLS])
df_test_scaled = df_test_engineered.copy()
df_test_scaled[CONTINUOUS_COLS] = final_scaler.transform(df_test_scaled[CONTINUOUS_COLS])

sub_indices = pd.read_csv('/kaggle/input/pirate/sample_submission.csv')['sample_index'].unique()
test_ds_final = PiratePainDataset(df_test_scaled, None, sub_indices, WINDOW_SIZE, STRIDE, TEXT_COL, augment=False)
test_loader_final = DataLoader(test_ds_final, batch_size=BATCH_SIZE*2, shuffle=False)
window_sample_map_test = [x[0] for x in test_ds_final.indices]

ensemble_logits = None

for i, model in enumerate(models_list):
    model.eval()
    fold_logits = []
    with torch.no_grad():
        for xc, xs, xt, xtxt, _ in test_loader_final:
            xc, xs, xt, xtxt = xc.to(device), xs.to(device), xt.to(device), xtxt.to(device)
            logits = model(xc, xs, xt, xtxt)
            fold_logits.extend(logits.cpu().numpy())
    
    df_tmp = pd.DataFrame(fold_logits, columns=[0, 1, 2])
    df_tmp['sample_index'] = window_sample_map_test
    df_avg = df_tmp.groupby('sample_index').mean()
    
    if ensemble_logits is None:
        ensemble_logits = df_avg
    else:
        ensemble_logits = ensemble_logits.add(df_avg, fill_value=0)

ensemble_logits /= K_FOLDS
final_probs = torch.softmax(torch.tensor(ensemble_logits.values), dim=1).numpy()

final_preds_list = []
thr_l, thr_h = best_thresh

for p in final_probs:
    if p[2] > thr_h: final_preds_list.append(2)
    elif p[1] > thr_l: final_preds_list.append(1)
    else: final_preds_list.append(0)

final_series = pd.Series(final_preds_list, index=ensemble_logits.index)

inv_map = {v: k for k, v in label_mapping.items()}
submission = final_series.map(inv_map).reset_index()
submission.columns = ['sample_index', 'label']

sample_sub = pd.read_csv('/kaggle/input/pirate/sample_submission.csv')
submission = submission.set_index('sample_index').reindex(sample_sub['sample_index']).reset_index()
submission.to_csv('submission.csv', index=False)
print("Fatto! Submission creata.")

Device in uso: cuda
--- 1. Caricamento e Pulizia Iniziale ---
Trovata colonna 'Team Name': n_legs
Calcolo Feature Engineering...

--- Avvio K-Fold con Ensemble Strategy ---

--- Fold 1/5 ---
Fold 1 Best Val F1: 0.9687

--- Fold 2/5 ---
Fold 2 Best Val F1: 0.9632

--- Fold 3/5 ---
Fold 3 Best Val F1: 0.9452

--- Fold 4/5 ---
Fold 4 Best Val F1: 0.9477

--- Fold 5/5 ---
Fold 5 Best Val F1: 0.9456

--- Ricerca Soglie Ottimali su OOF ---
Soglie Trovate: Low>0.45, High>0.45 -> OOF F1: 0.9542

--- Generazione Submission (Ensemble) ---
Fatto! Submission creata.
